In [84]:
import numpy as np
from scipy.special import gammaln
import matplotlib.pyplot as plt

## Collapsed Gibbs Sampling from scratch

From "Parameter estimation for text analysis", http://www.arbylon.net/publications/text-est.pdf

### Get Data

### Define algorithm

Initialize variables

In [66]:
dummy_input = np.array([
                       [0, 0, 1, 2, 2],
                       [0, 0, 1, 1, 1],
                       [0, 1, 2, 2, 2],
                       [4, 4, 4, 4, 4],
                       [3, 3, 4, 4, 4],
                       [3, 4, 4, 4, 4]])

n_topics = 4
n_docs, n_words = dummy_input.shape
doc_lens = np.sum(dummy_input, axis=1)

data = []

for i in range(n_docs):
    doc = []
    for j in range(n_words):
        doc += [j] *dummy_input[i, j]
    
    data += [doc]

beta = 1
alpha = np.ones(n_topics) * 1

# initialize topic assignments of words in each doc

topic_assignments = {}
for doc in range(n_docs):
    topic_assignments[doc] = [0] * doc_lens[doc]

cntTW = np.zeros((n_topics, n_words))          
cntDT = np.zeros((n_docs, n_topics))             
cntT = np.zeros(n_topics)  

In [8]:
np.array(data)

array([list([2, 3, 3, 4, 4]), list([2, 3, 4]),
       list([1, 2, 2, 3, 3, 4, 4]),
       list([0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4]),
       list([0, 0, 0, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4]),
       list([0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4])],
      dtype=object)

Begin iterating

In [67]:
for doc in range(n_docs):
    i = 0 # index for word position
    
    for word in data[doc]:
        topic = np.random.randint(0, n_topics-1)
        topic_assignments[doc][i] = topic
        cntTW[topic, word] += 1
        cntDT[doc, topic] += 1
        cntT[topic] += 1
        
        i+=1

In [74]:
def LogLikelihood():                                        # FIND (JOINT) LOG-LIKELIHOOD VALUE
    l = 0
    for z in range(n_topics):                                # log p(w|z,\beta)
        l += gammaln(n_words * beta)
        l -= n_words * gammaln(beta)
        l += np.sum(gammaln(cntTW[z] + beta))
        l -= gammaln(np.sum(cntTW[z] + beta))
    for doc in range(n_docs):                                  # log p(z|\alpha)
        l += gammaln(np.sum(alpha))
        l -= np.sum(gammaln(alpha))
        l += np.sum(gammaln(cntDT[doc] + alpha))
        l -= gammaln(np.sum(cntDT[doc] + alpha))
    return l

In [81]:
def findThetaPhi():
    th = np.zeros((n_docs, n_topics))                     # SPACE FOR THETA
    ph = np.zeros((n_topics, n_words))                   # SPACE FOR PHI
    for d in range(n_docs):
        for z in range(n_topics):
            th[d][z] = (cntDT[d][z] + alpha[z]) / (doc_lens[d] + np.sum(alpha))
    for z in range(n_topics):
        for w in range(n_words):
            ph[z][w] = (cntTW[z][w] + beta) / (cntT[z] + beta * n_words)
    return ph, th

Actual Gibbs Sampling

In [82]:
# COLLAPSED GIBBS SAMPLING
print("INITIAL STATE")
print("\tLikelihood:", LogLikelihood())               # FIND (JOINT) LOG-LIKELIHOOD

SAMPLES = 0
total_samples = 1000


burnin = 2000
interval = 10

phis = []
thetas = []

for s in range(burnin + total_samples):
    for doc in range(n_docs):
        i = 0 # index for word position
        for word in data[doc]:
            # decrement counts and sums for current assignment
            topic = topic_assignments[doc][i]
            cntTW[topic, word] -= 1
            cntDT[doc, topic] -= 1
            cntT[topic] -= 1    
            
            # calculate full conditional and sample new topic
            prL = (cntDT[doc] + alpha) / (doc_lens[doc] - 1 + np.sum(alpha))
            prR = (cntTW[:,word] + beta) / (cntT + beta * n_words)
            
            
            prFullCond = prL * prR  
            prFullCond = np.asarray(prFullCond)
            prFullCond /= np.sum(prFullCond)
            #print(prFullCond)
            #print(np.sum(prFullCond))
             
            new_topic = np.random.multinomial(1, prFullCond.astype(np.float64)).argmax()
            
            # update counts
            topic_assignments[doc][i] = new_topic
            cntTW[new_topic, word] += 1
            cntDT[doc, new_topic] += 1
            cntT[new_topic] += 1
            i+=1
            
      
    if s > burnin and s % interval == 0:                    # FIND PHI AND THETA AFTER BURN-IN POINT
        ph, th = findThetaPhi()
        thetas.append(th)
        phis.append(ph)
        print("\tLikelihood:", LogLikelihood())
        SAMPLES += 1
            

INITIAL STATE
	Likelihood: -212.46198132591644
	Likelihood: -208.8918553448105
	Likelihood: -202.944118158491
	Likelihood: -203.637363098979
	Likelihood: -201.19642732692444
	Likelihood: -218.18349515952139
	Likelihood: -210.97315045513966
	Likelihood: -192.78878040093895
	Likelihood: -191.77690763880872
	Likelihood: -217.42072921642864
	Likelihood: -207.67166829600137
	Likelihood: -205.18869938176402
	Likelihood: -209.20745222457663
	Likelihood: -211.08862789542263
	Likelihood: -201.41804441698162
	Likelihood: -216.5675986457219
	Likelihood: -197.9115467172603
	Likelihood: -189.44640143505865
	Likelihood: -195.52678738039987
	Likelihood: -193.76851509674748
	Likelihood: -212.92523920724284
	Likelihood: -204.07416881696463
	Likelihood: -208.97366923962448
	Likelihood: -195.18222896900522
	Likelihood: -208.12216808712822
	Likelihood: -200.70608329139674
	Likelihood: -211.83459508822392
	Likelihood: -204.9999528248136
	Likelihood: -201.25516925741323
	Likelihood: -209.46859217352602
	Lik